In [1]:
import pandas as pd
movies = pd.read_csv('moviedata_small/movies.csv')
person = pd.read_csv('moviedata_small/person.csv')
comments = pd.read_csv('moviedata_small/comments.csv')
users = pd.read_csv('moviedata_small/users.csv')
ratings = pd.read_csv('moviedata_small/ratings.csv')

### 打标签

In [3]:
import numpy as np
movies = pd.merge(movies,ratings,how='left',on='MOVIE_ID')
movies = movies.dropna(subset=['RATING'])
movies['labels'] = np.where(movies['RATING'] >= 3, 1, 0)

In [6]:
movies.to_csv('moviedata_backup/movies.csv')
movies_sample = movies.sample(n=20000, replace=False, random_state=100)
movies_sample.to_csv('moviedata_backup/movies_sample.csv')

In [1]:
import pandas as pd
movies = pd.read_csv('moviedata_backup/movies_sample.csv')

### 造特征

In [2]:
movie_feats = movies[['MOVIE_ID','NAME']]

## 电影演员个数
act_len = []
for act in movies['ACTORS'].str.split('/'):
    if act == act:
        act_len.append(len(act))
    else:
        act_len.append(0)
movie_feats['actors'] = act_len

## 电影导演个数
dire_len = []
for dire in movies['DIRECTORS'].str.split('/'):
    if dire == dire:
        dire_len.append(len(dire))
    else:
        dire_len.append(0)
movie_feats['directors'] = dire_len

## 电影类型 个数+one-hot编码
genres_len = []
for gen in movies['GENRES'].str.split('/'):
    if gen == gen:
        genres_len.append(len(gen))
    else:
        genres_len.append(0)
movie_feats['genres'] = genres_len

import numpy as np
genres = movies['GENRES'].str.split('/', expand=True).stack().reset_index(level=1, drop=True).rename('genres')
movie_tmp = movies[['MOVIE_ID']].join(genres)
genres_feats = pd.get_dummies(movie_tmp[['genres']])
movie_tmp = movie_tmp.drop(columns=['genres']).drop_duplicates()
movie_tmp = movie_tmp.join(genres_feats)
movie_tmp = movie_tmp.groupby('MOVIE_ID').sum()
movie_feats = pd.merge(movie_feats,movie_tmp,how='left',on='MOVIE_ID')

## 制片国家/地区 个数+one-hot编码
regions_len = []
for reg in movies['REGIONS'].str.split('/'):
    if reg == reg:
        regions_len.append(len(reg))
    else:
        regions_len.append(0)
movie_feats['regions'] = regions_len

# regions = movies['REGIONS'].str.split('/', expand=True).stack().reset_index(level=1, drop=True).rename('regions')
# movie_tmp = movies[['MOVIE_ID']].join(regions)
# regions_feats = pd.get_dummies(movie_tmp[['regions']])
# movie_tmp = movie_tmp.drop(columns=['regions']).drop_duplicates()
# movie_tmp = movie_tmp.join(regions_feats)
# movie_tmp = movie_tmp.groupby('MOVIE_ID').sum()
# movie_feats = pd.merge(movie_feats,movie_tmp,how='left',on='MOVIE_ID')


## 语言 个数
languages_len = []
for lg in movies['LANGUAGES'].str.split('/'):
    if lg == lg:
        languages_len.append(len(lg))
    else:
        languages_len.append(0)
movie_feats['languages'] = languages_len

## 语言 one-hot编码
# languages = movies['LANGUAGES'].str.split('/', expand=True).stack().reset_index(level=1, drop=True).rename('languages')
# movie_tmp = movies[['MOVIE_ID']].join(languages)
# languages_feats = pd.get_dummies(movie_tmp[['languages']])
# movie_tmp = movie_tmp.drop(columns=['languages']).drop_duplicates()
# movie_tmp = movie_tmp.join(languages_feats)
# movie_tmp = movie_tmp.groupby('MOVIE_ID').sum()
# movie_feats = pd.merge(movie_feats,movie_tmp,how='left',on='MOVIE_ID')

## 提取月、星期
movie_tmp = movies[['MOVIE_ID']]
movie_tmp['RELEASE_DATE'] = pd.to_datetime(movies['RELEASE_DATE'])
# movie_tmp['year'] = pd.DatetimeIndex(movie_tmp['RELEASE_DATE']).year
movie_tmp['month'] = pd.DatetimeIndex(movie_tmp['RELEASE_DATE']).month
movie_tmp['week'] = pd.DatetimeIndex(movie_tmp['RELEASE_DATE']).weekday
# year = pd.get_dummies(movie_tmp[['year']])
month = pd.get_dummies(movie_tmp[['month']])
week = pd.get_dummies(movie_tmp[['week']])
movie_tmp = movie_tmp.drop(columns=['RELEASE_DATE','month','week']).drop_duplicates()
# movie_tmp = movie_tmp.join(year)
movie_tmp = movie_tmp.join(month)
movie_tmp = movie_tmp.join(week)
movie_feats = pd.merge(movie_feats,movie_tmp,how='left',on='MOVIE_ID')

## 片长/豆瓣评分
movie_feats['mins'] = movies['MINS']
movie_feats['douban_score'] = movies['DOUBAN_SCORE']
movie_feats['douban_votes'] = movies['DOUBAN_VOTES']

## 标签 one-hot编码
# tags = movies['TAGS'].str.split('/', expand=True).stack().reset_index(level=1, drop=True).rename('tags')
# movie_tmp = movies[['MOVIE_ID']].join(tags)
# tags_feats = pd.get_dummies(movie_tmp[['tags']])
# movie_tmp = movie_tmp.drop(columns=['tags']).drop_duplicates()
# movie_tmp = movie_tmp.join(tags_feats)
# movie_tmp = movie_tmp.groupby('MOVIE_ID').sum()
# movie_feats = pd.merge(movie_feats,movie_tmp,how='left',on='MOVIE_ID')

tags_len = []
for tag in movies['TAGS'].str.split('/'):
    if tag == tag:
        tags_len.append(len(tag))
    else:
        tags_len.append(0)
movie_feats['tags'] = tags_len

/root/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/root/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/root/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

In [12]:
movie_feats

MOVIE_ID             NAME  actors  directors  genres  genres_Adult  \
0       1401265           第一类型危险      11          1       3             0   
1      25919961          蚱蜢 - 电影      11          1       1             0   
2      30327768            远去的牧歌       3          2       3             0   
3       7054604  变形金刚4：绝迹重生 - 电影      25          1       2             0   
4       1309023   午夜凶铃2(美版) - 电影      14          1       2             0   
...         ...              ...     ...        ...     ...           ...   
19995   1292329        牯岭街少年杀人事件      15          1       2             0   
19996   1301420             无名英雄       7          1       2             0   
19997   6849056        行歌坐月 - 电影       2          1       2             0   
19998   1428291        Jose与虎与鱼们      10          1       2             0   
19999  27042467             摇摆狂潮       8          1       3             0   

       genres_Comedy  genres_传记  genres_儿童  genres_冒险  ...  tags_（南瓜奇幻）  \
0                  0          0          0          0  ...            0   
1                  0          0          0          0  ...            0   
2                  0          0          0          1  ...            0   
3                  0          0          0          0  ...            0   
4                  0          0          0          0  ...            0   
...              ...        ...        ...        ...  ...          ...   
19995              0          0          0          0  ...            0   
19996              0          0          0          0  ...            0   
19997              0          0          0          0  ...            0   
19998              0          0          0          0  ...            0   
19999              0          0          0          0  ...            0   

       tags_（已收  tags_（待下  tags_（待）  tags_（恐怖）  tags_，小孩子的面部表情  tags_，歌舞不错，  \
0             0         0         0          0               0            0   
1             0         0         0          0               0            0   
2             0         0         0          0               0            0   
3             0         0         0          0               0            0   
4             0         0         0          0               0            0   
...         ...       ...       ...        ...             ...          ...   
19995         0         0         0          0               0            0   
19996         0         0         0          0               0            0   
19997         0         0         0          0               0            0   
19998         0         0         0          0               0            0   
19999         0         0         0          0               0            0   

       tags_？  tags_￥  tags  
0           0       0     8  
1           0       0     8  
2           0       0     8  
3           0       0     8  
4           0       0     8  
...       ...     ...   ...  
19995       0       0     8  
19996       0       0     8  
19997       0       0     8  
19998       0       0     8  
19999       0       0     8  

[20000 rows x 13558 columns]

## 电影描述 STORYLINE，word2vec自然语言处理

In [32]:
import jieba

# 读取停用词列表
def get_stopword_list(file):
    with open(file, 'r', encoding='utf-8') as f:    # 
        stopword_list = [word.strip('\n') for word in f.readlines()]
    return stopword_list


# 分词 然后清除停用词语
def clean_stopword(str, stopword_list):
    result = []
    word_list = jieba.lcut(str)   # 分词后返回一个列表  jieba.cut(）   返回的是一个迭代器
    for w in word_list:
        if w not in stopword_list:
            result.append(w)
    return result



In [4]:

if __name__ == '__main__':
    stopword_file = 'hit_stopwords.txt'
    stopword_list = get_stopword_list(stopword_file)    # 获得停用词列表
    seg_list=[]
    
    for line in movies['STORYLINE']:
#         print(line)
        if line == line:
            seg_list.append(clean_stopword(line, stopword_list))
        else:
            seg_list.append([])

from gensim.models import word2vec
# 训练模型
model = word2vec.Word2Vec(seg_list, vector_size=100)
# word_vectors = {}
# for word in model.wv.index_to_key:
#     word_vectors[word] = model.wv[word]     #获得词汇及其对应的向量

Building prefix dict from the default dictionary ...
2022-05-21 17:06:56,261 - jieba - DEBUG - Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
2022-05-21 17:06:57,454 - jieba - DEBUG - Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.281 seconds.
2022-05-21 17:06:57,544 - jieba - DEBUG - Loading model cost 1.281 seconds.
Prefix dict has been built successfully.
2022-05-21 17:06:57,546 - jieba - DEBUG - Prefix dict has been built successfully.


ModuleNotFoundError: No module named 'gensim'

--------------------
********************



异常信息: ModuleNotFoundError: No module named 'gensim'
问题原因: 包未安装
解决方法: https://links.alipay.com/custom/snippet/5eb388c3c92e8d8d9cfd3b00/latest/



********************
--------------------


### 采用IDF平滑处理

In [33]:
import math
import os
import pickle

# idf值统计方法
def train_idf(doc_list):
    idf_dic = {}
    # 总文档数
    tt_count = len(doc_list)

    # 每个词出现的文档数
    for doc in doc_list:
        for word in set(doc):
            idf_dic[word] = idf_dic.get(word, 0.0) + 1.0

    # 按公式转换为idf值，分母加1进行平滑处理
    for k, v in idf_dic.items():
        idf_dic[k] = math.log(tt_count / (1.0 + v))

    # 对于没有在字典中的词，默认其仅在一个文档出现，得到默认idf值
    print("tt_count" + str(tt_count))
    default_idf = math.log(tt_count / (1.0))
    return idf_dic, default_idf

def save_obj(obj, name ):
    with open('./'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('./' + name + '.pkl', 'rb') as f:
        return pickle.load(f)


In [ ]:
doc_list = seg_list
idf_dic, default_idf = train_idf(doc_list)
save_obj(idf_dic, 'idf_dic')
# idf_dic = load_obj('idf_dic')
print(default_idf)

### TFIDF加权平均词向量

In [35]:
def averageVector(many_vectors, vector_weight, column_num):
    """
    求多个向量的权值向量
    :param many_vector:
    :column_num:向量列数
    :vector_weight:IDF权重
    """
    average_vector = []
    # print(vector_weight)
    for i in range(0, column_num, 1):
        average_vector.append(0)
    row_num = len(many_vectors)
    # 先求出各个列权重之和  后面再求平均值
    row_index = 0
    for weight_index, vector in enumerate(many_vectors):
        for i in range(0, column_num, 1):
            average_vector[i] += float(vector[i]) * vector_weight[weight_index]
        row_index += 1
    for i in range(0, column_num, 1):
        average_vector[i] = average_vector[i] / row_num
    return average_vector

# 根据word2vec和tfidf获取句子矩阵
def get_sentence_matrix(splited_words):
    sentences_matrix = []
    sentences_matrix_str = []
    index = 0
    # 平均特征矩阵
    while index < len(splited_words):
        words_matrix = []
        word_weight = []
#         words = splited_words[index].split(" ")
        words = splited_words[index]
        # 得出各个词的特征向量  并形成一个矩阵  然后计算平均值  就得到该句子的特征向量
        for word in words:
            if word in model.wv.index_to_key:
                words_matrix.append(np.array(model.wv[word]))
                if word in idf_dic:
                    word_weight.append(idf_dic[word])
                else:
                    word_weight.append(default_idf)

#         print('>>>>>>>>>>words_matrix',words_matrix)
        if words_matrix != []:
            # 将words_matrix求均值
            feature = averageVector(many_vectors=words_matrix, vector_weight=word_weight,
                                    column_num= model.vector_size)
            sentences_matrix.append(feature)
            sentences_matrix_str.append(','.join(list(map(str,feature))))
        else:
            sentences_matrix.append([])
            sentences_matrix_str.append('')
        index += 1
    return sentences_matrix,sentences_matrix_str



In [21]:
sentences_matrix,sentences_matrix_str = get_sentence_matrix(seg_list)
movie_feats['storyline_emb'] = sentences_matrix_str


In [34]:
movie_feats.to_csv('moviedata_backup/movie_feats.csv')
feats = movie_feats.drop(columns=['MOVIE_ID','NAME'])
new_feats = movie_feats[['MOVIE_ID','NAME']]
new_feats['final_feats'] = feats['actors'].map(str)+','+feats['directors'].map(str)+','+feats['genres'].map(str)+','+feats['genres_Adult'].map(str)+','+feats['genres_Comedy'].map(str)+','+feats['genres_传记'].map(str)+','+feats['genres_儿童'].map(str)+','+feats['genres_冒险'].map(str)+','+feats['genres_剧情'].map(str)+','+feats['genres_动作'].map(str)+','+feats['genres_动画'].map(str)+','+feats['genres_历史'].map(str)+','+feats['genres_古装'].map(str)+','+feats['genres_同性'].map(str)+','+feats['genres_喜剧'].map(str)+','+feats['genres_奇幻'].map(str)+','+feats['genres_家庭'].map(str)+','+feats['genres_恐怖'].map(str)+','+feats['genres_悬念'].map(str)+','+feats['genres_悬疑'].map(str)+','+feats['genres_情色'].map(str)+','+feats['genres_惊悚'].map(str)+','+feats['genres_惊栗'].map(str)+','+feats['genres_戏曲'].map(str)+','+feats['genres_战争'].map(str)+','+feats['genres_歌舞'].map(str)+','+feats['genres_武侠'].map(str)+','+feats['genres_灾难'].map(str)+','+feats['genres_爱情'].map(str)+','+feats['genres_犯罪'].map(str)+','+feats['genres_真人秀'].map(str)+','+feats['genres_短片'].map(str)+','+feats['genres_科幻'].map(str)+','+feats['genres_纪录片'].map(str)+','+feats['genres_脱口秀'].map(str)+','+feats['genres_舞台艺术'].map(str)+','+feats['genres_西部'].map(str)+','+feats['genres_运动'].map(str)+','+feats['genres_音乐'].map(str)+','+feats['genres_驚悚 Thriller'].map(str)+','+feats['genres_鬼怪'].map(str)+','+feats['genres_黑色电影'].map(str)+','+feats['regions'].map(str)+','+feats['languages'].map(str)+','+feats['month'].map(str)+','+feats['week'].map(str)+','+feats['mins'].map(str)+','+feats['douban_score'].map(str)+','+feats['douban_votes'].map(str)+','+feats['tags'].map(str)+','+feats['storyline_emb'].map(str)
new_feats.to_csv('moviedata_backup/new_feats.csv')

/root/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [31]:
# sql = ''
# j=0
# for i in list(feats.keys()):
#     sql += '+feats[\''+i+'\'].map(str)+\',\''

# sql

"feats['actors'].map(str)+','+feats['directors'].map(str)+','+feats['genres'].map(str)+','+feats['genres_Adult'].map(str)+','+feats['genres_Comedy'].map(str)+','+feats['genres_传记'].map(str)+','+feats['genres_儿童'].map(str)+','+feats['genres_冒险'].map(str)+','+feats['genres_剧情'].map(str)+','+feats['genres_动作'].map(str)+','+feats['genres_动画'].map(str)+','+feats['genres_历史'].map(str)+','+feats['genres_古装'].map(str)+','+feats['genres_同性'].map(str)+','+feats['genres_喜剧'].map(str)+','+feats['genres_奇幻'].map(str)+','+feats['genres_家庭'].map(str)+','+feats['genres_恐怖'].map(str)+','+feats['genres_悬念'].map(str)+','+feats['genres_悬疑'].map(str)+','+feats['genres_情色'].map(str)+','+feats['genres_惊悚'].map(str)+','+feats['genres_惊栗'].map(str)+','+feats['genres_戏曲'].map(str)+','+feats['genres_战争'].map(str)+','+feats['genres_歌舞'].map(str)+','+feats['genres_武侠'].map(str)+','+feats['genres_灾难'].map(str)+','+feats['genres_爱情'].map(str)+','+feats['genres_犯罪'].map(str)+','+feats['genres_真人秀'].map(str)+','+feats

In [1]:
import pandas as pd
person = pd.read_csv('moviedata_small/person.csv')
comments = pd.read_csv('moviedata_small/comments.csv')
users = pd.read_csv('moviedata_small/users.csv')
ratings = pd.read_csv('moviedata_small/ratings.csv')

### Person数据

In [73]:
actors = movies['ACTORS'].str.split('/', expand=True).stack().reset_index(level=1, drop=True).rename('actors')
actors = list(set(actors.to_list()))
actors_type = ['演员']*len(actors)

directors = movies['DIRECTORS'].str.split('/', expand=True).stack().reset_index(level=1, drop=True).rename('directors')
directors = list(set(directors.to_list()))
directors_type = ['导演']*len(directors)

names = actors + directors
ntype = actors_type + directors_type
person_df = pd.DataFrame(names,columns=['NAME'])
person_df['type'] = ntype
persons = pd.merge(person,person_df,how='left',on='NAME')
persons = persons.dropna(subset=['type'])

In [81]:
## 人员角色
type_feats = pd.get_dummies(persons[['type']])
person_tmp = persons[['PERSON_ID','NAME']]
person_feats = person_tmp.join(type_feats)

## 性别
sex_feats = pd.get_dummies(persons[['SEX']])
person_feats = person_feats.join(sex_feats)

## 年龄
person_tmp['year'] = persons['BIRTH'].str.split('-',expand=True).loc[:,0]
person_tmp['year'] = person_tmp[['year']].fillna('2020')

year = []
for s in person_tmp['year']:
    if s.isdigit():
        year.append(int(s))
    else:
        year.append(2020)
person_tmp['age'] = 2019-np.array(year)

def age_map(age):
    if age>=1 and age<=7:return '1'
    if age>=8 and age<=16:return '2'
    if age>=17 and age<=29:return '3'
    if age>=30 and age<=39:return '4'
    if age>=40 and age<=49:return '5'
    if age>=50 and age<=59:return '6'
    if age>60:return '7'
    
person_feats['age'] = person_tmp['age'].apply(lambda age:age_map(age))
age_feats = pd.get_dummies(person_feats[['age']])
person_feats = person_feats.join(age_feats)

# ## 出生地
# birplace = persons['BIRTHPLACE'].str.split(',',expand=True)
# person_tmp['BIRTHPLACE'] = birplace.loc[:,0]+','+birplace.loc[:,1]
# place_feats = pd.get_dummies(person_tmp[['BIRTHPLACE']])
# person_feats = person_feats.join(place_feats)

## 星座
xz_feats = pd.get_dummies(persons[['CONSTELLATORY']])
person_feats = person_feats.join(xz_feats)

## 职业 个数
job_len = []
for job in persons['PROFESSION'].str.split('/'):
    if job == job:
        job_len.append(len(job))
    else:
        job_len.append(0)
person_feats['jobs'] = job_len

jobs = persons['PROFESSION'].str.split('/', expand=True).stack().reset_index(level=1, drop=True).rename('jobs')
person_tmp = persons[['PERSON_ID']].join(jobs)
jobs_feats = pd.get_dummies(person_tmp[['jobs']])
person_tmp = person_tmp.drop(columns=['jobs']).drop_duplicates()
person_tmp = person_tmp.join(jobs_feats)
person_tmp = person_tmp.groupby('PERSON_ID').sum()
person_feats = pd.merge(person_feats,person_tmp,how='left',on='PERSON_ID')



/root/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/root/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/root/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

In [66]:
# birth = []
# birth_place = []
# for s in persons['BIRTHPLACE']:
#     if s==s:
#         if '18' in s or '19' in s or '20' in s:
#             birth.append(s)
#             birth_place.append(None)
#         else:
#             birth.append(None)
#             birth_place.append(s)
#     else:
#         birth.append(None)
#         birth_place.append(None)
# for i in birth_place:
#     if i:
#         print(i)

In [79]:
job_feats

PROFESSION_主持人  PROFESSION_主持人 / 导演 / 制片人  PROFESSION_主持人 / 演员  \
0                   0                          0                    0   
1                   0                          0                    0   
2                   0                          0                    0   
3                   0                          0                    0   
4                   0                          0                    0   
...               ...                        ...                  ...   
74010               0                          0                    0   
74011               0                          0                    0   
74018               0                          0                    0   
74024               0                          0                    0   
74027               0                          0                    0   

       PROFESSION_主持人 / 演员 / 导演  PROFESSION_主持人 / 演员 / 导演 / 编剧  \
0                             0                              0   
1                             0                              0   
2                             0                              0   
3                             0                              0   
4                             0                              0   
...                         ...                            ...   
74010                         0                              0   
74011                         0                              0   
74018                         0                              0   
74024                         0                              0   
74027                         0                              0   

       PROFESSION_主持人 / 演员 / 配音  PROFESSION_主持人 / 演员 / 配音 / 导演  \
0                             0                              0   
1                             0                              0   
2                             0                              0   
3                             0                              0   
4                             0                              0   
...                         ...                            ...   
74010                         0                              0   
74011                         0                              0   
74018                         0                              0   
74024                         0                              0   
74027                         0                              0   

       PROFESSION_其它 / 动作特技 / 演员 / 导演 / 副导演  PROFESSION_其它 / 导演  \
0                                         0                   0   
1                                         0                   0   
2                                         0                   0   
3                                         0                   0   
4                                         0                   0   
...                                     ...                 ...   
74010                                     0                   0   
74011                                     0                   0   
74018                                     0                   0   
74024                                     0                   0   
74027                                     0                   0   

       PROFESSION_其它 / 导演 / 制片人  ...  PROFESSION_音乐 / 演员 / 配音  \
0                             0  ...                        0   
1                             0  ...                        0   
2                             0  ...                        0   
3                             0  ...                        0   
4                             0  ...                        0   
...                         ...  ...                      ...   
74010                         0  ...                        0   
74011                         0  ...                        0   
74018                         0  ...                        0   
74024                         0  ...                        0   
74027                   

#### 简介 BIOGRAPHY，w2v自然语言处理

In [82]:
stopword_file = 'hit_stopwords.txt'
stopword_list = get_stopword_list(stopword_file)    # 获得停用词列表
person_seg_list=[]

for line in persons['BIOGRAPHY']:
    if line == line:
        person_seg_list.append(clean_stopword(line, stopword_list))
    else:
        person_seg_list.append([])
        
# 训练模型
from gensim.models import word2vec
model = word2vec.Word2Vec(person_seg_list, vector_size=100)

doc_list = person_seg_list
idf_dic, default_idf = train_idf(doc_list)
save_obj(idf_dic, 'person_idf_dic')
# idf_dic = load_obj('idf_dic')
print(default_idf)

person_matrix,person_matrix_str = get_sentence_matrix(person_seg_list)
person_feats['biography_emb'] = person_matrix_str

2022-04-27 11:33:33,998 - gensim.models.word2vec - INFO - collecting all words and their counts
2022-04-27 11:33:34,000 - gensim.models.word2vec - INFO - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-04-27 11:33:34,044 - gensim.models.word2vec - INFO - PROGRESS: at sentence #10000, processed 73410 words, keeping 18226 word types
2022-04-27 11:33:34,111 - gensim.models.word2vec - INFO - PROGRESS: at sentence #20000, processed 170220 words, keeping 31627 word types
2022-04-27 11:33:34,184 - gensim.models.word2vec - INFO - PROGRESS: at sentence #30000, processed 299840 words, keeping 44253 word types
2022-04-27 11:33:34,243 - gensim.models.word2vec - INFO - collected 55926 word types from a corpus of 444360 raw words and 34772 sentences
2022-04-27 11:33:34,244 - gensim.models.word2vec - INFO - Creating a fresh vocabulary
2022-04-27 11:33:34,354 - gensim.utils - INFO - Word2Vec lifecycle event {'msg': 'effective_min_count=5 retains 9828 unique words (17.57322175732

tt_count34772
10.456567744198955


In [86]:
person_feats.to_csv('moviedata_backup/person_feats.csv')

In [84]:
# person_feats.head()
# sql = ''
# j=0
# for i in list(feats.keys()):
#     sql += '+feats[\''+i+'\'].map(str)+\',\''

# sql

PERSON_ID      NAME  type_导演  type_演员  SEX_女  SEX_男   age  age_1  age_2  \
0    1054424       刘德华        0        1      0      1     6      0      0   
1    1422441      杰夫·陈        1        0      0      1  None      0      0   
2    1422420  诺厄·勒·赫罗斯        0        1      0      1  None      0      0   
3    1422421  穆罕默德·卡里姆        0        1      0      1     4      0      0   
4    1422427      本·迪基        0        1      0      1  None      0      0   

   age_3  ...  jobs_编剧   jobs_美术   jobs_视觉特效   jobs_选角   jobs_配音  jobs_配音   \
0      0  ...         0         0           0         0        0         0   
1      0  ...         0         0           0         0        0         0   
2      0  ...         0         0           0         0        0         0   
3      0  ...         0         0           0         0        0         0   
4      0  ...         0         0           0         0        0         0   

   jobs_音乐  jobs_音乐   jobs_音效   \
0        0         0         0   
1        0         0         0   
2        0         0         0   
3        0         0         0   
4        0         0         0   

                                       biography_emb  
0  -1.3962045945640031,1.663098182364276,-0.64781...  
1                                                     
2                                                     
3                                                     
4                                                     

[5 rows x 99 columns]

In [88]:
feats = person_feats.drop(columns=['PERSON_ID','NAME'])
new_feats = person_feats[['PERSON_ID','NAME']]
new_feats['final_feats'] = feats['type_导演'].map(str)+','+feats['type_演员'].map(str)+','+feats['SEX_女'].map(str)+','+feats['SEX_男'].map(str)+','+feats['age'].map(str)+','+feats['age_1'].map(str)+','+feats['age_2'].map(str)+','+feats['age_3'].map(str)+','+feats['age_4'].map(str)+','+feats['age_5'].map(str)+','+feats['age_6'].map(str)+','+feats['age_7'].map(str)+','+feats['CONSTELLATORY_双子座'].map(str)+','+feats['CONSTELLATORY_双鱼座'].map(str)+','+feats['CONSTELLATORY_处女座'].map(str)+','+feats['CONSTELLATORY_天秤座'].map(str)+','+feats['CONSTELLATORY_天蝎座'].map(str)+','+feats['CONSTELLATORY_射手座'].map(str)+','+feats['CONSTELLATORY_巨蟹座'].map(str)+','+feats['CONSTELLATORY_摩羯座'].map(str)+','+feats['CONSTELLATORY_水瓶座'].map(str)+','+feats['CONSTELLATORY_狮子座'].map(str)+','+feats['CONSTELLATORY_白羊座'].map(str)+','+feats['CONSTELLATORY_金牛座'].map(str)+','+feats['jobs'].map(str)+','+feats['jobs_ 主持人'].map(str)+','+feats['jobs_ 主持人 '].map(str)+','+feats['jobs_ 其它'].map(str)+','+feats['jobs_ 其它 '].map(str)+','+feats['jobs_ 制片人'].map(str)+','+feats['jobs_ 制片人 '].map(str)+','+feats['jobs_ 制片管理'].map(str)+','+feats['jobs_ 制片管理 '].map(str)+','+feats['jobs_ 剪辑'].map(str)+','+feats['jobs_ 剪辑 '].map(str)+','+feats['jobs_ 副导演'].map(str)+','+feats['jobs_ 副导演 '].map(str)+','+feats['jobs_ 动作捕捉'].map(str)+','+feats['jobs_ 动作捕捉 '].map(str)+','+feats['jobs_ 动作特技'].map(str)+','+feats['jobs_ 动作特技 '].map(str)+','+feats['jobs_ 动画'].map(str)+','+feats['jobs_ 动画 '].map(str)+','+feats['jobs_ 化妆'].map(str)+','+feats['jobs_ 化妆 '].map(str)+','+feats['jobs_ 导演'].map(str)+','+feats['jobs_ 导演 '].map(str)+','+feats['jobs_ 摄影'].map(str)+','+feats['jobs_ 摄影 '].map(str)+','+feats['jobs_ 服装'].map(str)+','+feats['jobs_ 服装 '].map(str)+','+feats['jobs_ 演员'].map(str)+','+feats['jobs_ 演员 '].map(str)+','+feats['jobs_ 现场特效'].map(str)+','+feats['jobs_ 编剧'].map(str)+','+feats['jobs_ 编剧 '].map(str)+','+feats['jobs_ 美术'].map(str)+','+feats['jobs_ 美术 '].map(str)+','+feats['jobs_ 视觉特效'].map(str)+','+feats['jobs_ 视觉特效 '].map(str)+','+feats['jobs_ 选角'].map(str)+','+feats['jobs_ 选角 '].map(str)+','+feats['jobs_ 配音'].map(str)+','+feats['jobs_ 配音 '].map(str)+','+feats['jobs_ 音乐'].map(str)+','+feats['jobs_ 音乐 '].map(str)+','+feats['jobs_ 音效'].map(str)+','+feats['jobs_ 音效 '].map(str)+','+feats['jobs_主持人'].map(str)+','+feats['jobs_主持人 '].map(str)+','+feats['jobs_其它 '].map(str)+','+feats['jobs_制片人 '].map(str)+','+feats['jobs_制片管理 '].map(str)+','+feats['jobs_剪辑 '].map(str)+','+feats['jobs_副导演 '].map(str)+','+feats['jobs_动作特技'].map(str)+','+feats['jobs_动作特技 '].map(str)+','+feats['jobs_动画 '].map(str)+','+feats['jobs_化妆 '].map(str)+','+feats['jobs_导演'].map(str)+','+feats['jobs_导演 '].map(str)+','+feats['jobs_摄影'].map(str)+','+feats['jobs_摄影 '].map(str)+','+feats['jobs_服装 '].map(str)+','+feats['jobs_演员'].map(str)+','+feats['jobs_演员 '].map(str)+','+feats['jobs_编剧'].map(str)+','+feats['jobs_编剧 '].map(str)+','+feats['jobs_美术 '].map(str)+','+feats['jobs_视觉特效 '].map(str)+','+feats['jobs_选角 '].map(str)+','+feats['jobs_配音'].map(str)+','+feats['jobs_配音 '].map(str)+','+feats['jobs_音乐'].map(str)+','+feats['jobs_音乐 '].map(str)+','+feats['jobs_音效 '].map(str)+','+feats['biography_emb'].map(str)
new_feats.to_csv('moviedata_backup/person_str_feats.csv')

/root/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [26]:
movie_feats=pd.read_csv('moviedata_backup/movie_str_feats.csv')
person_feats=pd.read_csv('moviedata_backup/person_str_feats.csv')

In [25]:
movie_feats.head()

Unnamed: 0  new_id  movie_id             name  \
0           0       0   1401265           第一类型危险   
1           1       1  25919961          蚱蜢 - 电影   
2           2       2  30327768            远去的牧歌   
3           3       3   7054604  变形金刚4：绝迹重生 - 电影   
4           4       4   1309023   午夜凶铃2(美版) - 电影   

                                         final_feats  
0  11,1,3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0...  
1  11,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0...  
2  3,2,3,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...  
3  25,1,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0...  
4  14,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0...

### Comment数据

In [23]:
import word2vec

In [24]:
import pandas as pd
# from gensim.models import word2vec
comments = pd.read_csv('moviedata_small/comments.csv')
users = pd.read_csv('moviedata_small/users.csv')
ratings = pd.read_csv('moviedata_small/ratings.csv')

In [2]:
from gensim.models import word2vec

RuntimeError: Compiled extensions are unavailable. If you've installed from a package, ask the package maintainer to include compiled extensions. If you're building Gensim from source yourself, install Cython and a C compiler, and then run `python setup.py build_ext --inplace` to retry. 

In [27]:
movie_feats=movie_feats.rename(columns={'movie_id':'MOVIE_ID'})
movie_feats

Unnamed: 0  new_id  MOVIE_ID             name  \
0               0       0   1401265           第一类型危险   
1               1       1  25919961          蚱蜢 - 电影   
2               2       2  30327768            远去的牧歌   
3               3       3   7054604  变形金刚4：绝迹重生 - 电影   
4               4       4   1309023   午夜凶铃2(美版) - 电影   
...           ...     ...       ...              ...   
19995       19995   19995   1292329        牯岭街少年杀人事件   
19996       19996   19996   1301420             无名英雄   
19997       19997   19997   6849056        行歌坐月 - 电影   
19998       19998   19998   1428291        Jose与虎与鱼们   
19999       19999   19999  27042467             摇摆狂潮   

                                             final_feats  
0      11,1,3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0...  
1      11,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0...  
2      3,2,3,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...  
3      25,1,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0...  
4      14,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0...  
...                                                  ...  
19995  15,1,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0...  
19996  7,1,2,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...  
19997  2,1,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...  
19998  10,1,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0...  
19999  8,1,3,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...  

[20000 rows x 5 columns]

In [28]:
comments_tmp = pd.merge(comments,movie_feats[['MOVIE_ID','name']],how='left',on='MOVIE_ID')
comments = comments_tmp.dropna(subset=['name'])

In [ ]:
len(comments)   #4428475

In [29]:
comments_feats = comments[['COMMENT_ID','MOVIE_ID','USER_MD5','RATING','VOTES']]

## 评论时间
comments_tmp = pd.DataFrame(pd.to_datetime(comments['COMMENT_TIME']),columns=['COMMENT_TIME'])
year = pd.DatetimeIndex(comments_tmp['COMMENT_TIME']).year
month = pd.DatetimeIndex(comments_tmp['COMMENT_TIME']).month
week = pd.DatetimeIndex(comments_tmp['COMMENT_TIME']).weekday
year_feats = pd.get_dummies(year)
month_feats = pd.get_dummies(month)
week_feats = pd.get_dummies(week)
week_feats=week_feats.rename(columns={0:'w0',1:'w1',2:'w2',3:'w3',4:'w4',5:'w5',6:'w6'})

comments_tmp = comments_tmp.join(year_feats)
comments_tmp = comments_tmp.join(month_feats)
comments_tmp = comments_tmp.join(week_feats)

comments_tmp = comments_tmp.drop(columns=['COMMENT_TIME']).drop_duplicates()
comments_tmp['COMMENT_ID'] = comments[['COMMENT_ID']]
comments_feats = pd.merge(comments_feats,comments_tmp,how='left',on='COMMENT_ID')

#### CONTENT评论内容，自然语言处理

In [36]:
stopword_file = 'hit_stopwords.txt'
stopword_list = get_stopword_list(stopword_file)    # 获得停用词列表
content_seg_list=[]

for line in comments['CONTENT']:
    if line == line:
        content_seg_list.append(clean_stopword(line, stopword_list))
    else:
        content_seg_list.append([])


Building prefix dict from the default dictionary ...
2022-05-22 13:50:12,394 - jieba - DEBUG - Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
2022-05-22 13:50:13,868 - jieba - DEBUG - Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.740 seconds.
2022-05-22 13:50:14,136 - jieba - DEBUG - Loading model cost 1.740 seconds.
Prefix dict has been built successfully.
2022-05-22 13:50:14,139 - jieba - DEBUG - Prefix dict has been built successfully.


In [2]:
from gensim.models import word2vec
model = word2vec.Word2Vec(content_seg_list, vector_size=100)

doc_list = content_seg_list
idf_dic, default_idf = train_idf(doc_list)
save_obj(idf_dic, 'content_idf_dic')
# idf_dic = load_obj('idf_dic')
print(default_idf)

content_matrix,content_matrix_str = get_sentence_matrix(content_seg_list)
content_feats['content_emb'] = content_matrix_str

NameError: name 'content_seg_list' is not defined

--------------------
********************



异常信息: NameError: name 'content_seg_list' is not defined
问题原因: 代码书写问题
解决方法: 根据报错信息排查代码本身的问题



********************
--------------------


In [ ]:
content_feats.to_csv('moviedata_backup/content_feats.csv')
feats = content_feats.drop(columns=['COMMENT_ID','MOVIE_ID','USER_MD5'])
new_feats = content_feats[['COMMENT_ID','MOVIE_ID','USER_MD5']]

In [ ]:
# sql = ''
# j=0
# for i in list(feats.keys()):
#     sql += '+feats[\''+i+'\'].map(str)+\',\''

# sql

In [ ]:
content_feats

note：人（演员、导演）、观众、电影、评论

link：演员-导演、电影-演员、电影-导演、电影-观众（评分）、电影-评论、观众-评论

### 点表

In [11]:
movie_nodes = movies[['MOVIE_ID']]
mtype = [0]*len(movie_nodes)

person_nodes = person[['PERSON_ID']]
ptype = [1]*len(person_nodes)

user_nodes = users[['USER_MD5']]
utype = [2]*len(user_nodes)

comment_nodes = comments[['COMMENT_ID']]
ctype = [3]*len(comment_nodes)

movie_nodes

'王博:|吴佳尼:1313262|王姬:1275275|高丽雯:1325661|郭力行:1354456|尹哲:1326188|沈丹萍:1009141|罗中旭:1328354|臧金生:1045901|罗刚:1315029|居文沛:1275019|阎青妤:1317117'

'谷尚蔚/吴孟达/曾志伟/杜海涛'

In [13]:
movies['ACTOR_IDS'][1]

'谷尚蔚:1330813|吴孟达:1016771|曾志伟:1002862|杜海涛:1313024'

In [21]:
person[person['PERSON_ID']=='1016771']

/root/miniconda3/lib/python3.6/site-packages/pandas/core/ops/array_ops.py:252: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


Empty DataFrame
Columns: [PERSON_ID, NAME, SEX, NAME_EN, NAME_ZH, BIRTH, BIRTHPLACE, CONSTELLATORY, PROFESSION, BIOGRAPHY]
Index: []

### 边表

In [128]:
import jieba
import jieba.posseg as pseg
s="朝鲜半岛西北部古元古代高温变质-深熔作用:宏观和微观岩石学以及锆石U-Pb年代学制约"
 
#分词
jieba.cut(s, cut_all=False)
#分词并标注词性
segs = pseg.cut(s)
 
final = ''
for seg,flag in segs:
    #去数词和去字符串
    if flag !='m' and flag !='x':
        #输出分词
        final +=' '+ seg 
        #输出分词带词性
        # final +=' '+ seg+'/'+flag
print(final)

Building prefix dict from the default dictionary ...
2022-04-22 11:10:33,175 - jieba - DEBUG - Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
2022-04-22 11:10:35,053 - jieba - DEBUG - Dumping model to file cache /tmp/jieba.cache
Loading model cost 2.131 seconds.
2022-04-22 11:10:35,308 - jieba - DEBUG - Loading model cost 2.131 seconds.
Prefix dict has been built successfully.
2022-04-22 11:10:35,313 - jieba - DEBUG - Prefix dict has been built successfully.


 朝鲜半岛 西北部 古 元古代 高温 变质 深 熔 作用 宏观 和 微观 岩石学 以及 锆石 Pb 年代学 制约


In [141]:
# link_n1 = new_l1n1+new_l2n1+new_l3n1+new_l4n1
# link_n2 = new_l1n2+new_l2n2+new_l3n2+new_l4n2
# link_ltype = [0]*len(new_l1n1)+[1]*len(new_l2n1)+[2]*len(new_l3n1)+[3]*len(new_l4n1)
# feats = ['']*len(new_l1n1)+['']*len(new_l2n1)+['']*len(new_l3n1)+['']*len(new_l4n1)

# import pandas as pd
# links = pd.DataFrame()
# links['node_id1']=link_n1
# links['node_id2']=link_n2
# links['link_type']=link_ltype
# links['feats']=feats

import pandas as pd
link1 = pd.DataFrame()
link1['node_id1']=new_l1n1
link1['node_id2']=new_l1n2
link1['link_type']=[0]*len(new_l1n1)
link1['feats']=['']*len(new_l1n1)

link2 = pd.DataFrame()
link2['node_id1']=new_l2n1
link2['node_id2']=new_l2n2
link2['link_type']=[1]*len(new_l2n1)
link2['feats']=['']*len(new_l2n1)

link3 = pd.DataFrame()
link3['node_id1']=new_l3n1
link3['node_id2']=new_l3n2
link3['link_type']=[2]*len(new_l3n1)
link3['feats']=['']*len(new_l3n1)

link4 = pd.DataFrame()
link4['node_id1']=new_l4n1
link4['node_id2']=new_l4n2
link4['link_type']=[3]*len(new_l4n1)
link4['feats']=['']*len(new_l4n1)

### 挑选子图node

In [142]:
paper = set()
paper.update(tuple(new_l2n2))
paper.update(tuple(new_l3n1))
paper.update(tuple(new_l3n2))
paper.update(tuple(new_l4n1))
paper = list(sorted(paper))

author = set()
author.update(tuple(new_l1n1))
author.update(tuple(new_l2n1))
author = list(sorted(author))

institution = set()
institution.update(tuple(new_l1n2))
institution = list(sorted(institution))

field_of_study = set()
field_of_study.update(tuple(new_l4n2))
field_of_study = list(sorted(field_of_study))

In [143]:
num_paper = len(paper)
num_author = len(author)
num_institution = len(institution)
num_field_of_study = len(field_of_study)

nodeid_paper = list(range(num_paper))
nodeid_author = list(range(max(nodeid_paper)+1,max(nodeid_paper) + num_author + 1))
nodeid_institution = list(range(max(nodeid_author)+1,max(nodeid_author) + num_institution + 1))
nodeid_field_of_study = list(range(max(nodeid_institution)+1,max(nodeid_institution) + num_field_of_study + 1))
nodeid = nodeid_paper+nodeid_author+nodeid_institution+nodeid_field_of_study

nodename = paper+author+institution+field_of_study
nodetype = [0]*num_paper+[1]*num_author+[2]*num_institution+[3]*num_field_of_study

import pandas as pd
node = pd.DataFrame()
node['node_id']=nodeid
node['node_name']=nodename
node['node_type']=nodetype


feats_a = ['']*num_author
feats_t = ['']*num_institution
feats_c = ['']*num_field_of_study

new_feats_p = graph['node_feat_dict']['paper'][paper]
feats_p = []
for fs in new_feats_p:
    feats_p.append(','.join(list(map(str,fs))))
    
node['feats']=feats_p+feats_a+feats_t+feats_c

In [57]:
node.head(100)

node_id  node_name  node_type  \
0         0          0          0   
1         1     524288          0   
2         2     524290          0   
3         3          3          0   
4         4          5          0   
..      ...        ...        ...   
95       95     524443          0   
96       96        153          0   
97       97        156          0   
98       98        159          0   
99       99     524448          0   

                                                feats  
0   -0.095379,0.040758,-0.210948,-0.064362,-0.2259...  
1   -0.03265,-0.231902,0.056324,-0.381083,-0.00136...  
2   -0.142372,-0.044907,-0.263803,-0.124775,-0.170...  
3   0.004506,0.042368,-0.178465,-0.138479,-0.13826...  
4   -0.314425,-0.126466,-0.068305,-0.035875,0.1978...  
..                                                ...  
95  -0.008216,0.178697,-0.150037,-0.066784,0.17875...  
96  0.14886,-0.126798,-0.204972,-0.014371,-0.01355...  
97  -0.108812,-0.042026,-0.331045,-0.099419,-0.022...  
98  -0.076158,-0.036799,-0.15098,0.018213,0.13025,...  
99  -0.155013,-0.204861,0.001981,-0.179237,0.05991...  

[100 rows x 4 columns]

### 重新编号link_id

In [144]:
node_a = node[node['node_type'] == 1][['node_id','node_name']].rename(columns={'node_name':'node_id1'})
new_link1_id1 = pd.merge(link1,node_a,how='left',on='node_id1')['node_id'].tolist()  
node_i = node[node['node_type'] == 2][['node_id','node_name']].rename(columns={'node_name':'node_id2'})
new_link1_id2 = pd.merge(link1,node_i,how='left',on='node_id2')['node_id'].tolist()  

new_link2_id1 = pd.merge(link2,node_a,how='left',on='node_id1')['node_id'].tolist()  
node_p = node[node['node_type'] == 0][['node_id','node_name']].rename(columns={'node_name':'node_id2'})
new_link2_id2 = pd.merge(link2,node_p,how='left',on='node_id2')['node_id'].tolist()  

new_link3_id2 = pd.merge(link3,node_p,how='left',on='node_id2')['node_id'].tolist()  
node_p = node[node['node_type'] == 0][['node_id','node_name']].rename(columns={'node_name':'node_id1'})
new_link3_id1 = pd.merge(link3,node_p,how='left',on='node_id1')['node_id'].tolist()  

new_link4_id1 = pd.merge(link4,node_p,how='left',on='node_id1')['node_id'].tolist()  
node_f = node[node['node_type'] == 3][['node_id','node_name']].rename(columns={'node_name':'node_id2'})
new_link4_id2 = pd.merge(link4,node_f,how='left',on='node_id2')['node_id'].tolist()  

link1['new_node_id1']=new_link1_id1
link1['new_node_id2']=new_link1_id2

link2['new_node_id1']=new_link2_id1
link2['new_node_id2']=new_link2_id2

link3['new_node_id1']=new_link3_id1
link3['new_node_id2']=new_link3_id2

link4['new_node_id1']=new_link4_id1
link4['new_node_id2']=new_link4_id2

links = pd.concat([link1,link2,link3,link4],axis=0)

### 重新编号label_id

In [158]:
import pandas as pd
labels = pd.DataFrame()
labels['node_name'] = label_idx[0]
labels['label'] = label['paper'][label_idx[0]]
node_p = node[node['node_type'] == 0]
labels = pd.merge(labels,node_p,how='left',on='node_name')

### 保存文件

In [148]:
file_name = 'dataset/data/node.dat'
with open(file_name, "w") as f:
        for nid, nname, ntype, fts in node.values:
            f.write(f"{int(nid)}\t{nname}\t{int(ntype)}\t{fts}\n")

In [149]:
file_name = 'dataset/data/link.dat'
with open(file_name, "w") as f:
            for old_nid1, old_nid2, ltype, fts, nid1, nid2 in links.values:
                f.write(f"{int(nid1)}\t{int(nid2)}\t{int(ltype)}\t{fts}\n")

In [162]:
l = np.array(range(len(labels)))
np.random.shuffle(l)
split = int(len(l)*0.7)
labels_test = labels.loc[l[:split]]
labels_train = labels.loc[l[split:]]

In [164]:
file_name = 'dataset/data/label.dat'
with open(file_name, "w") as f:
            for nname, label, nid, ntype, feats in labels_train.values:
                f.write(f"{int(nid)}\t{nname}\t{int(ntype)}\t{label}\n")
                
file_name = 'dataset/data/label.dat.test'
with open(file_name, "w") as f:
            for nname, label, nid, ntype, feats in labels_test.values:
                f.write(f"{int(nid)}\t{nname}\t{int(ntype)}\t{label}\n")

In [4]:
ss = ''
i=0.11
while i<=0.5:
    ss += ',('+str(i)+',1)'
    i += 0.001
    i = round(i,3)
ss

',(0.11,1),(0.111,1),(0.112,1),(0.113,1),(0.114,1),(0.115,1),(0.116,1),(0.117,1),(0.118,1),(0.119,1),(0.12,1),(0.121,1),(0.122,1),(0.123,1),(0.124,1),(0.125,1),(0.126,1),(0.127,1),(0.128,1),(0.129,1),(0.13,1),(0.131,1),(0.132,1),(0.133,1),(0.134,1),(0.135,1),(0.136,1),(0.137,1),(0.138,1),(0.139,1),(0.14,1),(0.141,1),(0.142,1),(0.143,1),(0.144,1),(0.145,1),(0.146,1),(0.147,1),(0.148,1),(0.149,1),(0.15,1),(0.151,1),(0.152,1),(0.153,1),(0.154,1),(0.155,1),(0.156,1),(0.157,1),(0.158,1),(0.159,1),(0.16,1),(0.161,1),(0.162,1),(0.163,1),(0.164,1),(0.165,1),(0.166,1),(0.167,1),(0.168,1),(0.169,1),(0.17,1),(0.171,1),(0.172,1),(0.173,1),(0.174,1),(0.175,1),(0.176,1),(0.177,1),(0.178,1),(0.179,1),(0.18,1),(0.181,1),(0.182,1),(0.183,1),(0.184,1),(0.185,1),(0.186,1),(0.187,1),(0.188,1),(0.189,1),(0.19,1),(0.191,1),(0.192,1),(0.193,1),(0.194,1),(0.195,1),(0.196,1),(0.197,1),(0.198,1),(0.199,1),(0.2,1),(0.201,1),(0.202,1),(0.203,1),(0.204,1),(0.205,1),(0.206,1),(0.207,1),(0.208,1),(0.209,1),(0.21,1),

In [53]:
from ogb import nodeproppred #import DglNodePropPredDataset

dataset = nodeproppred.NodePropPredDataset(name = 'ogbn-arxiv')

split_idx = dataset.get_idx_split()
train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]
graph, label = dataset[0] # graph: dgl graph object, label: torch tensor of shape (num_nodes, num_tasks)

Downloaded 0.08 GB: 100%|██████████| 81/81 [00:09<00:00,  8.36it/s]


Extracting dataset/arxiv.zip
Loading necessary files...
This might take a while.


100%|██████████| 1/1 [00:00<00:00, 9822.73it/s]

Processing graphs...
Saving...


In [2]:
# -*- coding: utf-8 -*-
import oss2

auth = oss2.Auth('LTAIUZP8NYWRKcxj', 'HgX2YJDxuRIYht6f41XhgqlRdHUIBn')
bucket = oss2.Bucket(auth, 'http://oss-cn-hangzhou.aliyuncs.com', 'kk2019')

# 上传文件到OSS。
# <yourObjectName>由包含文件后缀，不包含Bucket名称组成的Object完整路径，例如abc/efg/123.jpg。
# <yourLocalFile>由本地文件路径加文件名包括后缀组成，例如/users/local/myfile.txt。
bucket.put_object_from_file('resource/cb60f2dbdd04020bf5b0c425e38a156e0.image', 'cb60f2dbdd04020bf5b0c425e38a156e0.jpg')

FileNotFoundError: [Errno 2] No such file or directory: 'cb60f2dbdd04020bf5b0c425e38a156e0.jpg'

In [15]:
import csv
import requests
import os
import json
import sys

# language=CH
# endpoint=oss-cn-shanghai-alipay-mybank-internal.aliyun-inc.com
# accessKeyID=LTAI5tFBZAsVZ4DaeeMfv3au
# accessKeySecret=7LYzJuK4e88UsENz5kzKro0fWMNE8B

data_dir = 'zz_image'
save_dir = 'image'


def download_csv(csv_path, image_column_index):
    with open(csv_path, 'r', encoding='UTF-8') as f:
        lines = f.read().splitlines()
        for line in lines[1:]:
            row = line.split(',')
#             print(row)
            if row[0] == 'e.其他(大于万5）':
                image_column_index=2
                url = row[image_column_index]
            else:
                image_column_index=3
                url = row[image_column_index]
            print(url)
            image_download(url)


def image_download(url):
    save_path = f'{save_dir}/{data_dir}/images/'
    cmd = f'ossutil64 cp oss://bksecossbkmmdcmybank/{url} {save_path} -f -c siyi.ossutilconfig'
    os.system(cmd)


if __name__ == '__main__':
    # custom
#     image_column_index = 3
    csv_path = f'{save_dir}/{data_dir}.csv'

    os.makedirs(f'{save_dir}/{data_dir}/images/', exist_ok=True)
    download_csv(csv_path, image_column_index)

biz/screditgrowup/20220401/6e2a9bbe1ae0c96ca26c63b4ec57d21c.JPEG
biz/screditgrowup/20220324/34f8b41a0340110f9ed71583aee49824.JPEG
biz/screditgrowup/20220320/05f391eeb4a0be9180c5127b83943aa0.JPEG
biz/screditgrowup/20220327/78cf59afa50caf7683fa31302ea68fcb.jpg
biz/screditgrowup/20220403/f4271c0ca54dfb6927df25e6a01df9fb.PNG
biz/screditgrowup/20220401/c0f9a0758bfaeb915b66a21b8d1a4faa.jpg
biz/screditgrowup/20220329/e3a5707fa5651c5281e6efb8fb615aec.JPEG
biz/screditgrowup/20220410/4cf65f33e0336e67e69b4c11e40a87bb.jpg
biz/screditgrowup/20220321/e7e0d7dfaa698975d46c6038f6b632f8.jpg
biz/screditgrowup/20220320/2d75e0e3a90e7639325ce0373f6a99b1.jpg
biz/screditgrowup/20220318/1474d176e09e533cc5cb994deb997a3a.jpg
biz/screditgrowup/20220401/91d2c9fe71f7c8f94322c4d70f7807cb.jpg
biz/screditgrowup/20220326/c7a88d7cff982306c5bddc8c07ea6aef.JPEG
biz/screditgrowup/20220321/761ba4b129cdcdb41dc3a1ed21010204.JPEG
biz/screditgrowup/20220401/29df6d549ac05a3909c3a024ca88acf9.jpg
biz/screditgrowup/20220402/e2147b5

In [18]:
from pypai.model import upload_model
dd = upload_model('image/zz_image/images.zip', name="images")

model saved, id = 2841161
model saved at oss path :  oss://cmps-model/pai/model/221530/1649923795374605/py_model.tar.gz
download model url :  http://cmps-model.cn-hangzhou.alipay.aliyun-inc.com/pai/model/221530/1649923795374605/py_model.tar.gz


In [19]:
dd

(2841161, 'oss://cmps-model/pai/model/221530/1649923795374605/py_model.tar.gz')